# Cell 1 - Imports & paths

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from pathlib import Path

# Local imports (repo root assumed)
import sys
repo_root = Path("..").resolve()  # notebooks/ is one level down
sys.path.append(str(repo_root / "src"))

from utils import sanity_check_soc_timeseries, pool_consistency_check

In [ ]:
# Cell 2 - Load data

In [ ]:
data_dir = repo_root / "data" / "synthetic"

soc_timeseries = pd.read_csv(data_dir / "soc_timeseries.csv")
soc_pools = pd.read_csv(data_dir / "soc_pools.csv")
sensitivity = pd.read_csv(data_dir / "sensitivity_results.csv")

soc_timeseries.head(), soc_pools.head(), sensitivity.head()

# Cell 3 - Sanity checks

In [ ]:
checks = sanity_check_soc_timeseries(soc_timeseries)
checks

# Cell 4 — Plot SOC trajectories (baseline vs project)

In [ ]:
for scenario in soc_timeseries["scenario"].unique():
    df_s = soc_timeseries[soc_timeseries["scenario"] == scenario]
    plt.plot(df_s["year"], df_s["total_soc_tCha"], label=scenario)

plt.xlabel("Year")
plt.ylabel("Total SOC (t C / ha)")
plt.title("Total SOC trajectory (synthetic) — baseline vs project")
plt.legend()
plt.show()

# Cell 5 — Year-on-year ΔSOC

In [ ]:
for scenario in soc_timeseries["scenario"].unique():
    df_s = soc_timeseries[soc_timeseries["scenario"] == scenario]
    plt.plot(df_s["year"], df_s["delta_soc_tCha_per_yr"], label=scenario)

plt.axhline(0, linewidth=1)
plt.xlabel("Year")
plt.ylabel("ΔSOC (t C / ha / yr)")
plt.title("Year-on-year SOC change (synthetic)")
plt.legend()
plt.show()

# Cell 6 — Pool-level plots (quick scan)

In [ ]:
pools = soc_pools["pool"].unique()

for pool in pools:
    df_p = soc_pools[soc_pools["pool"] == pool]
    for scenario in df_p["scenario"].unique():
        df_ps = df_p[df_p["scenario"] == scenario]
        plt.plot(df_ps["year"], df_ps["soc_tCha"], label=f"{pool}-{scenario}")
    plt.xlabel("Year")
    plt.ylabel("SOC (t C / ha)")
    plt.title(f"Pool dynamics — {pool} (synthetic)")
    plt.legend()
    plt.show()

# Cell 7 — Pool consistency check

In [ ]:
consistency = pool_consistency_check(soc_pools, soc_timeseries)
consistency.head(10)

In [ ]:
# Quick summary: max relative diff
consistency["rel_diff"].max()

# Cell 8 — Sensitivity table

In [ ]:
sensitivity.sort_values("rank")

In [ ]:
plt.bar(sensitivity["parameter"], sensitivity["impact"])
plt.xticks(rotation=30, ha="right")
plt.ylabel("Relative impact")
plt.title("Synthetic sensitivity ranking (RothC-like drivers)")
plt.show()

# Cell 9 — Fitness-for-use decision matrix 

## Fitness-for-use decision matrix

**Decision rule (conservative):**
If uncertainty materially overlaps with zero sequestration, results must be downscaled, caveated, or excluded from external claims.

| Use case | Fit? | Rationale |
|---|---|---|
| Internal exploration |  |  |
| External reporting |  |  |
| Verification & audit |  |  |

**Key blockers (if any):**
- …